In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr 
from pyspark.testing import assertDataFrameEqual
from pyspark.sql.types import StructType, StructField

In [3]:
spark = SparkSession.builder.appName("test").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/20 11:47:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [58]:
m = spark.read.csv("merchants.csv", header=True, inferSchema=True) 
t = spark.read.csv("transactions.csv", header=True, inferSchema=True)  

In [62]:
SELECT distinct fraudbits FROM "transactions.csv" 

,fraudbits
0,0
1,1


In [63]:
WITH operation_efficiency AS (
    SELECT
    m.guild AS guild,
    SUM(CASE WHEN t.countertype = 'sell' THEN 1 ELSE 0 END) / SUM(CASE WHEN t.countertype = 'buy' THEN 1 ELSE 0 END) AS sale_buy_ratio,
    AVG(ABS(t.cost - t.model_cost) / m.coef) AS average_amnesty
    FROM "transactions.csv" t INNER JOIN "merchants.csv" m ON t.merchantid = m.merchantid
    WHERE t.fraudbits = 0
    GROUP BY m.guild
)

SELECT guild, ROUND(average_amnesty * sale_buy_ratio, 2) AS efficiency, sale_buy_ratio, average_amnesty 
FROM operation_efficiency
ORDER BY efficiency DESC
-- LIMIT 1;

,guild,efficiency,sale_buy_ratio,average_amnesty
0,Братство Кузнецов,196.36,0.106963,1835.734533
1,Братство Виноделов,193.03,0.111808,1726.452617
2,Объединение Фруктовых Садовников,154.03,0.110625,1392.373808
3,Гильдия Обработчиков Шерсти,145.23,0.107331,1353.118922
4,Союз Каменотесов,126.41,0.125455,1007.618129
5,Гильдия Пекарей,116.60,0.113617,1026.286880
6,Лига Керамистов,105.41,0.111820,942.637071
7,Гильдия Августовских Торговцев,99.69,0.105953,940.918641
8,Гильдия Парфюмеров,96.87,0.104556,926.507174
9,Союз Судостроителей,90.63,0.122625,739.047568


In [67]:
SELECT 
    SUBSTRING(t.date FROM 11 FOR 2) AS hour,
    COUNT(*) AS sales_count
FROM "transactions.csv" t INNER JOIN "merchants.csv" m ON t.merchantid = m.merchantid
WHERE t.countertype = 'sell' AND m.guild = 'Братство Кузнецов'
GROUP BY SUBSTRING(t.date FROM 11 FOR 2)
ORDER BY sales_count DESC
-- LIMIT 1;

,hour,sales_count
0,21,48
1,13,44
2,20,43
3,17,40
4,08,39
5,10,38
6,19,36
7,14,35
8,15,34
9,07,34
